In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as nnf
from torchsummary import summary
import torchvision


In [4]:
class resenet_block(nn.Module):
    def __init__(self,channels):
        super().__init__()
        self.conv1 = nn.Conv2d(channels,channels,3,padding='same')
        self.bn1 = nn.BatchNorm2d(channels)
        self.conv2 = nn.Conv2d(channels,channels,3,padding='same')
        self.bn2 = nn.BatchNorm2d(channels)

    def forward(self,x):
        y = self.conv1(x)
        y = nnf.relu(self.bn1(y))
        y = self.conv2(y)
        y = nnf.relu(self.bn2(x+y))
        return y
        


class Resnet(nn.Module):
    def __init__(self, in_channels, output_dimension=1,downscale_architecture='resnet34'):
        super().__init__()
        self.downscale_architecture = downscale_architecture
        self.model_layers = []
        if self.downscale_architecture == 'resnet34':
            self.downscale_architecture = [3,3,5,3]
            
        self.model_layers.append(nn.Conv2d(in_channels,64,7,padding='same'))
        self.model_layers.append(nn.BatchNorm2d(64))
        self.model_layers.append(nn.ReLU())
        #add relu
        self.model_layers.append(nn.MaxPool2d(3,stride=2,padding=1))
        in_channels = 64



        for count,resnet_section_length in enumerate(self.downscale_architecture):
            for i in range(resnet_section_length):
                self.model_layers.append(resenet_block(in_channels))
            if count < (len(self.downscale_architecture)-1):
                self.model_layers.append(nn.Conv2d(in_channels,in_channels*2,kernel_size=3,stride=2,padding=1))
                in_channels = in_channels*2
                self.model_layers.append(nn.BatchNorm2d(in_channels))
                self.model_layers.append(nn.ReLU())
                self.model_layers.append(nn.Conv2d(in_channels,in_channels,kernel_size=3,stride=1,padding='same'))
                self.model_layers.append(nn.BatchNorm2d(in_channels))
                self.model_layers.append(nn.ReLU())

        self.model_layers.append(nn.AdaptiveAvgPool2d((1,1)))
        #add flatten
        self.model_layers.append(nn.Flatten())
        self.model_layers.append(nn.Linear(in_channels,1000))
        self.model_layers.append(nn.ReLU())
        self.model_layers.append(nn.Linear(1000,output_dimension))

        self.model = nn.Sequential(*self.model_layers)

    def forward(self,x):
        return self.model(x)


    def model_train(self,dataloader,epochs,lr=2e-4):
        self.model.train()
        criterion_loss = nn.MSELoss()
        optimizer = torch.optim.Adam(self.model.parameters(), lr = lr)

        for i, data in enumerate(dataloader):

            x = data['x']
            y = data['y']

            self.model.zero_grad()

            y_pred = self.model(x)
            loss = criterion_loss(y_pred,y)

            loss.backward()
            optimizer.step()

In [30]:

transform = torchvision.transforms.Compose([torchvision.transforms.ToTensor(),
                                            torchvision.transforms.Normalize(0.5, 0.5)])
batch_size  = 100 
dataset = torchvision.datasets.MNIST(root='\\data',download=True,transform=transform)
params = {'batch_size': batch_size,
          'shuffle': True}


loader = torch.utils.data.DataLoader(dataset,**params)

In [35]:
res = Resnet(1,10,[1,1,1,1])
res.model.train()
criterion_loss = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(res.model.parameters(), lr = 2e-4)

while(1):
    for i, data in enumerate(loader):

        x = data[0]
        y = data[1]

        res.model.zero_grad()

        y_pred = res.model(x)
        loss = criterion_loss(y_pred,y)

        loss.backward()
        optimizer.step()

        if not i%10:
            print(loss.item())


2.3102662563323975
0.9343811273574829
0.30711138248443604
0.2920389473438263
0.11648187786340714
0.2307494729757309
0.07715723663568497
0.1767752468585968
0.06582262367010117
0.16501714289188385


KeyboardInterrupt: 

In [26]:
torch.zeros((32,78))

torch.float32